In [ ]:
#Gerekli kütüphanelerin import edilmesi
import warnings
import itertools
from math import sqrt
from datetime import datetime
from numpy import concatenate
import numpy as np
import pandas as pd
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM, Bidirectional, GRU
from keras.layers.recurrent import LSTM
from sklearn.utils import shuffle

import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
py.init_notebook_mode(connected=True)
plt.style.use('fivethirtyeight')

In [ ]:
# Datanın Yüklenmesi
data = pd.read_excel('dataset.xlsx', date_parser=[0])

# İlk 5 Satır
print(data.head())
#Son 5 satır
print(data.tail())

In [ ]:
#Datetime Haline Getirilmesi
data['tarih'] = pd.to_datetime(data.tarih, format='%d-%m-%Y')

#İndex'e Alınması
data.index = data.tarih

#Bilindiği gibi cumartesi pazar piyasalar kapalı ve burada da o yüzden o datalar NaN gözüküyor. 
#Bunların doldurulması için interpolate fonksiyonunu kullanabiliriz. 
#Bu fonksiyon haftaya bakarak lineer olarak dolduruyor.
data['usd_alis'].interpolate(method='linear', inplace=True)

In [ ]:
#Doların 2000 Ekim-2019 Mart arasındaki değişimi
fig = plt.figure(figsize=(15,8))
data.usd_alis.plot(label='usd alış')
plt.legend(loc='best')
plt.title('Günlük Döviz Alış Kurları', fontsize=14)
plt.show()

In [ ]:
#Verinin derin öğrenme modellerine ve LSTM’e verilmeden önce 0-1 arasınaa scale edilmesi.
#Bu scale işlemi yapılmadan da model çalışabilir fakat scale edilme işlemi sonrası elde edilen sonuçlar ciddi oranda
#daha başarılı
values = data['usd_alis'].values.reshape(-1,1)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(values)

# Birkaç Değere Bakalım
dataset[0:5]

In [ ]:
#Veri setinin eğitim ve test diye 2 ye bölünmesi. %80 Eğitim % 20 Test
TRAIN_SIZE = 0.80

train_size = int(len(dataset) * TRAIN_SIZE)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print("Gün Sayıları (eğitim seti, test seti): " + str((len(train), len(test))))


In [ ]:
#Veri setinin adım adım okunması için ön işleme adımları 
#Buradaki mantık son güne bakıp diğer günün tahmin edilmesi şeklinde olacak.
#window_size=1  1 gün önceye bakma şeklinde. Başka denemeler için parametrik
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [ ]:
# Verisetlerimizi Oluşturalım
window_size = 1
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)
print("Orijinal eğitim veri seti şekli:")
print(train_X.shape)

# Yeni verisetinin şekline bakalım.
#The LSTM giriş katmanı 3D olmak zorunda.Bu 3 boyut: örnek sayısı, zaman aralığı ve özellikler.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("Yeni eğitim veri seti şekli:")
print(train_X.shape)

In [ ]:
#Şimdi ise LSTM Modelimizi kuralım.
def fit_model(train_X, train_Y, window_size = 1):
    #Bir girdi tensoru ve bir çıktı tensoru olduğu durumlarda en uygun seçim sequential
    model = Sequential() 
    # Model tek layerlı şekilde kurulacak.
    model.add(LSTM(100, input_shape = (1, window_size)))
    model.add(Dense(1)) #1 nöron içeren gizli katman
    model.compile(loss = "mean_squared_error", optimizer = "adam") #kullanılacak hata fonksiyonu ve optimizer(stochastic gradient descent metodu)
   #30 epoch yani 30 kere veri setine bakılacak.
    model.fit(train_X, 
              train_Y, 
              epochs = 30, 
              batch_size = 1, 
              verbose = 1)
    
    return(model)

# Modelin eğitim veri seti ile eğitilmesi
model1 = fit_model(train_X, train_Y, window_size)

In [ ]:
#Sonuçların ölçülmesi
def predict_and_score(model, X, Y):
    # Tahminlerin 0-1 ile scale edilmiş halinden geri çevrilmesi.
    pred = scaler.inverse_transform(model.predict(X))
    orig_data = scaler.inverse_transform([Y])
    # Rmse(hata) değerlerinin ölçülmesi
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return(score, pred)

rmse_train, train_predict = predict_and_score(model1, train_X, train_Y)
rmse_test, test_predict = predict_and_score(model1, test_X, test_Y)

print("Eğitim veri seti skoru: %.2f RMSE" % rmse_train)
print("Test veri seti skoru: %.2f RMSE" % rmse_test)

In [ ]:
#Elde edilen sonuçların görselleştirilmesi
# Öğrendiklerinin tahminletilmesi
train_predict_plot = np.empty_like(dataset)
train_predict_plot[:, :] = np.nan
train_predict_plot[window_size:len(train_predict) + window_size, :] = train_predict

# Testlerin tahminletilmesi
test_predict_plot = np.empty_like(dataset)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict) + (window_size * 2) + 1:len(dataset) - 1, :] = test_predict

# Grafik haline getirilmesi
plt.figure(figsize = (15, 5))
plt.plot(scaler.inverse_transform(dataset), label = "Gerçek Değer")
plt.plot(train_predict_plot, label = "Eğitim Veri Seti Tahmini")
plt.plot(test_predict_plot, label = "Test Veri Seti Tahmini")
plt.xlabel("Günler")
plt.ylabel("Döviz Kurları")
plt.title("Gerçek ve Tahmin Edilen Eğitim / Test Verileri Karşılaştırılması")
plt.legend()
plt.show()